# Backtesting with [**vectorbt**](https://github.com/polakowo/vectorbt)
* Data Acquisition (**yfinance**)
* Statistical and Technical Indicators (**polars-ti**)
* Backtesting Analysis and Results (**vectorbt**)

### Initializations

In [ ]:
import asyncio
import itertools
from datetime import datetime

from IPython import display

import numpy as np
import pandas as pd
import polars_ti as ti
import vectorbt as vbt

import plotly.graph_objects as go

print("Package Versions:")
print(f"Numpy v{np.__version__}")
print(f"Pandas v{pd.__version__}")
print(f"vectorbt v{vbt.__version__}")
print(
    f"\nPolars TI v{ti.version}\nTo install the Latest Version:\n$ pip install -U git+https://github.com/CMobley7/polars-ti\n"
)

%matplotlib inline

# **vectorbt** Theme and Portfolio Settings

In [ ]:
cheight, cwidth = 500, 1000  # Adjust as needed for Chart Height and Width
vbt.settings.set_theme("dark")  # Options: "light" (Default), "dark" (my fav), "seaborn"

# Must be set
vbt.settings.portfolio["freq"] = "1D"  # Daily

# Predefine vectorbt Portfolio settings
vbt.settings.portfolio["init_cash"] = 100_000
vbt.settings.portfolio["fees"] = 0.0025  # 0.25%
vbt.settings.portfolio["slippage"] = 0.0025  # 0.25%
# vbt.settings.portfolio["size"] = 100
# vbt.settings.portfolio["accumulate"] = False
vbt.settings.portfolio["allow_partial"] = False
vbt.settings.portfolio["signal_direction"] = "both"

pf_settings = pd.DataFrame(vbt.settings.portfolio.items(), columns=["Option", "Value"])
pf_settings.set_index("Option", inplace=True)

print(f"Portfolio Settings [Initial]")
pf_settings

## Helper Methods

In [ ]:
def combine_stats(
    pf: vbt.portfolio.base.Portfolio, ticker: str, strategy: str, mode: int = 0
):
    header = pd.Series(
        {
            "Run Time": ti.get_time(full=False, to_string=True),
            "Mode": "LIVE" if mode else "TEST",
            "Strategy": strategy,
            "Direction": vbt.settings.portfolio["signal_direction"],
            "Symbol": ticker.upper(),
            "Fees [%]": 100 * vbt.settings.portfolio["fees"],
            "Slippage [%]": 100 * vbt.settings.portfolio["slippage"],
            "Accumulate": vbt.settings.portfolio["accumulate"],
        }
    )
    rstats = pf.returns_stats().dropna(axis=0).T
    stats = pf.stats().dropna(axis=0).T
    joint = pd.concat([header, stats, rstats])
    return joint[~joint.index.duplicated(keep="first")]


def earliest_common_index(d: dict):
    """Returns index of the earliest common index of all DataFrames in the dict"""
    min_date = None
    for df in d.values():
        if min_date is None:
            min_date = df.index[0]
        elif min_date < df.index[0]:
            min_date = df.index[0]
    return min_date


def dl(tickers: list, same_start: bool = False, **kwargs):
    if isinstance(tickers, str):
        tickers = [tickers]

    if not isinstance(tickers, list) or len(tickers) == 0:
        print("Must be a non-empty list of tickers or symbols")
        return

    if "limit" in kwargs and kwargs["limit"] and len(tickers) > kwargs["limit"]:
        from itertools import islice

        tickers = list(islice(tickers, kwargs["limit"]))
        print(
            f"[!] Too many assets to compare. Using the first {kwargs['limit']}: {', '.join(tickers)}"
        )

    print(f"[i] Downloading: {', '.join(tickers)}")

    received = {}
    if len(tickers):
        _df = pd.DataFrame()
        for ticker in tickers:
            received[ticker] = _df.ti.ticker(ticker, **kwargs)
            print(f"[+] {ti.get_time(full=False, to_string=True)}")

    if same_start and len(tickers) > 1:
        earliestci = earliest_common_index(received)
        print(f"[i] Earliest Common Date: {earliestci}")
        result = {
            ticker: df[df.index > earliestci].copy() for ticker, df in received.items()
        }
    else:
        result = received
    print(f"[*] Download Complete\n")
    return result


def dtmask(df: pd.DataFrame, start: datetime, end: datetime):
    return df.loc[(df.index >= start) & (df.index <= end), :].copy()


def show_data(d: dict):
    [
        print(
            f"{t}[{df.index[0]} - {df.index[-1]}]: {df.shape} {df.ti.time_range:.2f} years"
        )
        for t, df in d.items()
    ]


def trade_table(pf: vbt.portfolio.base.Portfolio, k: int = 1, total_fees: bool = False):
    if not isinstance(pf, vbt.portfolio.base.Portfolio):
        return
    k = int(k) if isinstance(k, int) and k > 0 else 1

    df = pf.trades.records[
        [
            "status",
            "direction",
            "size",
            "entry_price",
            "exit_price",
            "return",
            "pnl",
            "entry_fees",
            "exit_fees",
        ]
    ]
    if total_fees:
        df["total_fees"] = df["entry_fees"] + df["exit_fees"]

    print(f"\nLast {k} of {df.shape[0]} Trades\n{df.tail(k)}\n")

# Data Acquisition

#### Specify Symbols for Benchmarks and Assets

In [ ]:
benchmark_tickers = ["SPY", "QQQ"]
asset_tickers = ["AAPL", "TSLA", "TWTR"]
all_tickers = benchmark_tickers + asset_tickers

print("Tickers by index #")
print("=" * 100)
print(f"Benchmarks: {', '.join([f'{k}: {v}' for k,v in enumerate(benchmark_tickers)])}")
print(f"    Assets: {', '.join([f'{k}: {v}' for k,v in enumerate(asset_tickers)])}")
print(f"       All: {', '.join([f'{k}: {v}' for k,v in enumerate(all_tickers)])}")

In [ ]:
benchmark = benchmark_tickers[0]  # Change index for different benchmark
asset = asset_tickers[2]  # Change index for different symbol
print(f"Selected Benchmark | Asset: {benchmark} | {asset}")

In [ ]:
benchmarks = dl(benchmark_tickers, timed=True)

In [ ]:
assets = dl(asset_tickers, timed=True)

### Define Testing Dates and Ranges

In [8]:
start_date = datetime(2005, 1, 1)  # Adjust as needed
end_date = datetime(2010, 1, 1)  # Adjust as needed

### Select and Benchmark and Asset to Backtest

In [ ]:
print("Available Data:")
print("=" * 100)
print(f"Benchmarks: {', '.join(benchmarks.keys())}")
print(f"Assets: {', '.join(assets.keys())}")

In [ ]:
benchmark_name = "SPY"  # Select a Benchmark
asset_name = "AAPL"  # Select an Asset

benchmarkdf = benchmarks[benchmark_name]
assetdf = assets[asset_name]

# Set True if you want to constrain Data between start_date & end_date
common_range = True
if common_range:
    crs = f" from {start_date} to {end_date}"
    benchmarkdf = dtmask(benchmarkdf, start_date, end_date)
    assetdf = dtmask(assetdf, start_date, end_date)

# Update DataFrame names
benchmarkdf.name = benchmark_name
assetdf.name = asset_name
print(
    f"Analysis of: {benchmarkdf.name} and {assetdf.name}{crs if common_range else ''}"
)

#### Sanity Check

In [ ]:
benchmarkdf

In [ ]:
assetdf

## Creating Trading Signals for **vectorbt**
**vectorbt** can create a Backtest using ```vbt.Portfolio.from_signals(*args, **kwargs)``` based on trends that you create with **Polars TI**.

### Trend Creation
A **Trend** is the result of some calculation or condition of one or more indicators. For simplicity, a _Trend_ is either ```True``` or ```1``` and _No Trend_ is ```False``` or ```0```. Using the **Hello World** of Trends, the **Golden/Death Cross**, it's Trend is _Long_ when ```long = ma(close, 50) > ma(close, 200) ``` and _Short_ when ```short = ma(close, 50) < ma(close, 200) ```. 

In [ ]:
# Example Long Trends for the selected Asset
# * Uncomment others for exploration or replace them with your own TI Trend Strategy
def trends(df: pd.DataFrame, mamode: str = "sma", fast: int = 50, slow: int = 200):
    return ti.ma(mamode, df.Close, length=fast) > ti.ma(
        mamode, df.Close, length=slow
    )  # SMA(fast) > SMA(slow) "Golden/Death Cross"


#     return ti.increasing(ti.ma(mamode, df.Close, length=fast)) # Increasing MA(fast)
#     return ti.macd(df.Close, fast, slow).iloc[:,1] > 0 # MACD Histogram is positive

### Display Trends

In [14]:
trend_kwargs = {"mamode": "ema", "fast": 20, "slow": 50}

In [ ]:
benchmark_trends = trends(benchmarkdf, **trend_kwargs)
benchmark_trends.copy().astype(int).plot(
    figsize=(16, 1),
    kind="area",
    color=["limegreen"],
    alpha=0.9,
    title=f"{benchmarkdf.name} Trends",
    grid=True,
).axhline(0, color="black")

In [ ]:
asset_trends = trends(assetdf, **trend_kwargs)
asset_trends.copy().astype(int).plot(
    figsize=(16, 1),
    kind="area",
    color=["limegreen"],
    alpha=0.98,
    title=f"{assetdf.name} Trends",
    grid=True,
).axhline(0, color="black")

### **Trend Signals** 
Given a _Trend_, **Trend Signals** returns the _Trend_, _Trades_, _Entries_ and _Exits_ as boolean integers. When ```asbool=True```, it returns _Trends_, _Entries_ and _Exits_ as boolean values which is helpful when combined with the [**vectorbt**](https://github.com/polakowo/vectorbt) backtesting package.

In [ ]:
# trade_offset = 0 for Live Signals (close is last price)
# trade_offset = 1 for Backtesting
LIVE = 0

benchmark_signals = assetdf.ti.tsignals(
    benchmark_trends, asbool=True, trade_offset=LIVE, append=True
)
benchmark_signals.tail()

In [ ]:
asset_signals = assetdf.ti.tsignals(
    asset_trends, asbool=True, trade_offset=LIVE, append=True
)
asset_signals.tail()

## Creating **vectorbt** Portfolios
* [**vectorbt** Portfolio Options](https://polakowo.io/vectorbt/docs/portfolio/base.html)

### Buy 'N Hold Portfolios with their Single Trade and Performance Statistics

In [ ]:
# Benchmark Buy and Hold (BnH) Strategy
benchmarkpf_bnh = vbt.Portfolio.from_holding(benchmarkdf.Close)
print(trade_table(benchmarkpf_bnh))
combine_stats(benchmarkpf_bnh, benchmarkdf.name, "Buy and Hold", LIVE)

In [ ]:
# Asset Buy and Hold (BnH) Strategy
assetpf_bnh = vbt.Portfolio.from_holding(assetdf.Close)
print(trade_table(assetpf_bnh))
combine_stats(assetpf_bnh, assetdf.name, "Buy and Hold", LIVE)

### Signal Portfolios with their Last 'k' Trades and Performance Statistics

In [ ]:
# Benchmark Portfolio from Trade Signals
benchmarkpf_signals = vbt.Portfolio.from_signals(
    benchmarkdf.Close,
    entries=benchmark_signals.TS_Entries,
    exits=benchmark_signals.TS_Exits,
)
trade_table(benchmarkpf_signals, k=5)
combine_stats(benchmarkpf_signals, benchmarkdf.name, "Long Strategy", LIVE)

In [ ]:
# Asset Portfolio from Trade Signals
assetpf_signals = vbt.Portfolio.from_signals(
    assetdf.Close,
    entries=asset_signals.TS_Entries,
    exits=asset_signals.TS_Exits,
)
trade_table(assetpf_signals, k=5)
combine_stats(assetpf_signals, assetdf.name, "Long Strategy", LIVE)

## Buy and Hold Plots

In [23]:
vbt.settings.set_theme("seaborn")

### Benchmark

In [ ]:
benchmarkpf_bnh.trades.plot(
    title=f"{benchmarkdf.name} | Trades", height=cheight, width=cwidth
).show_png()

In [ ]:
benchmarkpf_bnh.value().vbt.plot(
    title=f"{benchmarkdf.name} | Equity Curve",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
benchmarkpf_bnh.drawdown().vbt.plot(
    title=f"{benchmarkdf.name} | Drawdown",
    trace_kwargs=dict(name="%"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
benchmarkpf_bnh.trades.plot_pnl(
    title=f"{benchmarkdf.name} | PnL", height=cheight // 2, width=cwidth
).show_png()

In [ ]:
benchmarkpf_bnh.asset_returns().vbt.plot(
    title=f"{benchmarkdf.name} | Asset Returns",
    trace_kwargs=dict(name="%"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
benchmarkpf_bnh.cash().vbt.plot(
    title=f"{benchmarkdf.name} | Cash",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
total_assetfees = (
    benchmarkpf_bnh.trades.records_readable["Entry Fees"]
    + benchmarkpf_bnh.trades.records_readable["Exit Fees"]
)
total_assetfees.vbt.plot(
    title=f"{benchmarkdf.name} | Total Fees",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

### Asset

In [ ]:
assetpf_bnh.trades.plot(
    title=f"{assetdf.name} | Trades", height=cheight, width=cwidth
).show_png()

In [ ]:
assetpf_bnh.value().vbt.plot(
    title=f"{assetdf.name} | Equity Curve",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
assetpf_bnh.drawdown().vbt.plot(
    title=f"{assetdf.name} | Drawdown",
    trace_kwargs=dict(name="%"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
assetpf_bnh.trades.plot_pnl(
    title=f"{assetdf.name} | PnL", height=cheight // 2, width=cwidth
).show_png()

In [ ]:
assetpf_bnh.asset_returns().vbt.plot(
    title=f"{assetdf.name} | Asset Returns",
    trace_kwargs=dict(name="%"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
assetpf_bnh.cash().vbt.plot(
    title=f"{assetdf.name} | Cash",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
total_assetfees = (
    assetpf_bnh.trades.records_readable["Entry Fees"]
    + assetpf_bnh.trades.records_readable["Exit Fees"]
)
total_assetfees.vbt.plot(
    title=f"{assetdf.name} | Total Fees",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

## Signal Plots

In [38]:
vbt.settings.set_theme("dark")

### Benchmark

In [ ]:
benchmarkpf_signals.trades.plot(
    title=f"{benchmarkdf.name} | Trades", height=cheight, width=cwidth
).show_png()

In [ ]:
benchmarkpf_signals.value().vbt.plot(
    title=f"{benchmarkdf.name} | Equity Curve",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
benchmarkpf_signals.drawdown().vbt.plot(
    title=f"{benchmarkdf.name} | Drawdown",
    trace_kwargs=dict(name="%"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
benchmarkpf_signals.trades.plot_pnl(
    title=f"{benchmarkdf.name} | PnL", height=cheight // 2, width=cwidth
).show_png()

In [ ]:
benchmarkpf_signals.asset_returns().vbt.plot(
    title=f"{benchmarkdf.name} | Asset Returns",
    trace_kwargs=dict(name="%"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
benchmarkpf_signals.cash().vbt.plot(
    title=f"{benchmarkdf.name} | Cash",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
total_assetfees = (
    benchmarkpf_signals.trades.records_readable["Entry Fees"]
    + benchmarkpf_signals.trades.records_readable["Exit Fees"]
)
total_assetfees.vbt.plot(
    title=f"{benchmarkdf.name} | Total Fees",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

### Asset

In [ ]:
assetpf_signals.trades.plot(
    title=f"{assetdf.name} | Trades", height=cheight, width=cwidth
).show_png()

In [ ]:
assetpf_signals.value().vbt.plot(
    title=f"{assetdf.name} | Equity Curve",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
assetpf_signals.drawdown().vbt.plot(
    title=f"{assetdf.name} | Drawdown",
    trace_kwargs=dict(name="%"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
assetpf_signals.trades.plot_pnl(
    title=f"{assetdf.name} | PnL", height=cheight // 2, width=cwidth
).show_png()

In [ ]:
assetpf_signals.asset_returns().vbt.plot(
    title=f"{assetdf.name} | Asset Returns",
    trace_kwargs=dict(name="%"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
assetpf_signals.cash().vbt.plot(
    title=f"{assetdf.name} | Cash",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

In [ ]:
total_assetfees = (
    assetpf_signals.trades.records_readable["Entry Fees"]
    + assetpf_signals.trades.records_readable["Exit Fees"]
)
total_assetfees.vbt.plot(
    title=f"{assetdf.name} | Total Fees",
    trace_kwargs=dict(name="\u00a4"),
    height=cheight // 2,
    width=cwidth,
).show_png()

# Disclaimer
* All investments involve risk, and the past performance of a security, industry, sector, market, financial product, trading strategy, or individual’s trading does not guarantee future results or returns. Investors are fully responsible for any investment decisions they make. Such decisions should be based solely on an evaluation of their financial circumstances, investment objectives, risk tolerance, and liquidity needs.

* Any opinions, news, research, analyses, prices, or other information offered is provided as general market commentary, and does not constitute investment advice. I will not accept liability for any loss or damage, including without limitation any loss of profit, which may arise directly or indirectly from use of or reliance on such information.